In [1]:
import os
import pandas as pd
import numpy as np
import requests
import folium
import bs4
import json
from folium import plugins

In [2]:
csv = './test.csv'

df = pd.read_csv(csv, encoding = 'utf8')

In [3]:
m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

In [5]:
district_list = []
lat_list = []
long_list = []
api_key = "0C781246-6E25-31B6-966A-E2A614B069E1"

for addr in df['real_address']: #vworld api를 사용하여 위도 경도 추출
    res = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={addr}&refine=true&simple=false&format=xml&type=road&key={api_key}")
    soup = bs4.BeautifulSoup(res.content, 'html.parser')
    district_list.append(soup.find('level2').text)
    lat_list.append(soup.find('x').text)
    long_list.append(soup.find('y').text)
#     print('='*50)
#     print(soup.find('level2').text)
#     print(soup.find('x').text)
#     print(soup.find('y').text)
#     print('='*50)

In [7]:
df['district'] = district_list
df['latitude'] = lat_list
df['longitude'] = long_list

df['latitude'] = df['latitude'].astype('float64')
df['longitude'] = df['longitude'].astype('float64')

In [22]:
counts = df['district'].value_counts()
df = df[~df['district'].isin(counts[counts < 2].index)]

In [23]:
df['latitude'][0]

127.112611095

In [79]:
m = folium.Map(location = [37.562225, 126.978555],
               zoom_start=11)
folium.Marker(location = [df['latitude'][0],df['longitude'][0]],popup = df['name'][0], icon = folium.Icon(color='red', icon='star')).add_to(m)
for lat, long, name in zip(df['latitude'], df['longitude'], df['name']):
    popup = folium.Popup(name, max_width=2650)
    loc=[long,lat]
    folium.CircleMarker(loc, radius = 10, color = "red", fill_color="red", popup=popup).add_to(m)

In [80]:
m

In [96]:
import json
import folium
from folium.plugins import HeatMap

m2 = folium.Map(location = [37.562225, 126.978555],
               zoom_start=11)

heatMap = HeatMap(zip(df['longitude'], df['latitude']),
               min_opacity=0.2,
               max_val=10,
               radius=50, blur=20
               )

m2.add_child(heatMap)
m2

<ipython-input-96-dfef8e28183d>:8: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  heatMap = HeatMap(zip(df['longitude'], df['latitude']),
